# **Градиент и антиградиент**

В предыдущих юнитах мы научились дифференцировать различные функции, а также реализовывать простейшие алгоритмы оптимизации. Теперь пришло время подойти к самому известному алгоритму в сфере анализа данных — **градиентному спуску**. Он применяется почти в каждой модели машинного обучения, а также его используют в глубинном обучении для нейронных сетей в модификации, которую называют **обратным распространением ошибки**.

В следующем юните мы подробно разберём принцип работы данного алгоритма, а пока нам необходимо набраться терпения и создать фундамент для освоения этого метода. В первую очередь нужно разобраться с понятием **градиента** — центральным понятием в алгоритме градиентного спуска.

Давайте представим, что у нас есть функция, которая приближённо прогнозирует прибыль:

![](data/46.PNG)

Если мы захотим максимизировать доход, то, скорее всего, решим сформулировать задачу оптимизации, а затем перейдём к вычислению частных производных. Вы уже прекрасно умеете их искать — давайте убедимся в этом.



In [2]:
from sympy import *
x,y=symbols('x y' )
f = (x**2)*y+4*y+y*x
f_diff_x = f.diff(x)
f_diff_y = f.diff(y)
print(f_diff_x)
print(f_diff_y)

2*x*y + y
x**2 + x + 4


Теперь как будто бы надо научиться вычислять ещё и градиент. Но вы только что сделали это, сами того не заметив. Как же так?
***
* **Градиент функции** f, обозначаемый как **∇f**, представляет собой вектор, компоненты которого равны частным производным функции f по всем её аргументам:

![](https://lms.skillfactory.ru/assets/courseware/v1/8d1da4a82d256f2eb0c560c0069b531f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_1.png)

Таким образом, вы можете найти каждую производную отдельно, соединить все производные вместе и получить градиент.
***

Заметьте, что **∇f** — *векторнозначная* функция. В данном примере эта функция ставит в соответствие двум значениям из одного множества два значения из другого. Из этого следует, что эту функцию очень удобно визуализировать с помощью векторного поля, которое представляет собой множество векторов градиента, проведённых из всех возможных точек:

![](https://lms.skillfactory.ru/assets/courseware/v1/c5dcbd13794a21bc245d84771abb536c/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_3.png)

Для того чтобы получить такое поле, в каждой точке проводится вектор, который образуется, если подставить эту точку в вектор градиента. Это векторное поле ещё часто называют градиентным полем функции.

Разумеется, градиент можно найти не только для функции, которая зависит от двух аргументов. Попробуйте сделать это для функции трёх переменных ↓

In [6]:
# Найдите вектор градиента для функции f = x-xy+z^2:

from sympy import *
x,y,z=symbols('x y z')
f = x - x*y + z**2
f_diff_x = f.diff(x)
f_diff_y = f.diff(y)
f_diff_z = f.diff(z)
print(f_diff_x,"\n", f_diff_y,"\n", f_diff_z)

1 - y 
 -x 
 2*z


Отлично — мы научились находить градиент, но зачем он, собственно, нужен и чем так интересен?

Представим функцию в виде холмистой местности. Пусть мы стоим в какой-то точке этой местности (x1, y1). Если мы будем менять своё положение по оси x или по оси y в положительную сторону, то будем подниматься на холм. Но обычно при движении мы меняем координаты положения не по одному измерению, а все разом, то есть меняем положение в пространстве по градиенту. Именно здесь важно осознать самое главное свойство градиента, которое и сделало его столь популярным у аналитиков и специалистов по машинному обучению, — градиент определяет **направление наискорейшего роста функции**.

Давайте ещё раз обдумаем это утверждение. Стоит вспомнить определение производной, которое мы вводили в предыдущем модуле: производная определяет скорость роста функции при изменении какого-либо аргумента. В данном случае мы изменяем все аргументы разом, а значит ускорение придаётся со всех сторон.

Если мы заменим все координаты градиента на значения с противоположным знаком, то получим антиградиент, который является направлением наискорейшего убывания функции.

Проще представить это визуально. Допустим, график нашей функции выглядит, как на изображении ниже, то есть у него есть два максимума, которые визуально представлены двумя холмами.

![](https://lms.skillfactory.ru/assets/courseware/v1/18f51240da591ce1a84f25b56b37e060/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_7.png)

Тогда градиентное поле будет выглядеть следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/62f19e0850af68c5b5f1cd0907487584/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_8.png)

Внимательно присмотритесь к нему, и вы увидите, что, в какую бы точку плоскости мы ни попали, там оказывается стрелка, которая указывает в сторону ближайшего холма.

Если очень сильно упростить, именно так устроен алгоритм градиентного спуска, но подробнее об этом мы поговорим в следующем юните.

Чтобы понять ещё одно свойство градиента, вспомним о том, что также градиентом называют плавный переход от одного цвета к другому. Это выглядит следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/40b15ce784126dce26aba484bb0c94d5/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_9.png)

Перекладывая понимание градиента на изменение оттенка, а не значения функции (хотя на самом деле можно сказать, что изменяется значение функции, выражающей оттенок), можно утверждать, что на данном рисунке:

* Градиент — это стрелка, задающая направление заливки.
* Линии, вдоль которых яркость одинаковая, будут ортогональны градиенту. Назовём их линиями уровня яркости.
* Двигаясь вдоль градиента, мы максимально скоро выйдем на самое яркое место.

Данный пример нужен нам для того, чтобы перейти к пониманию линий уровня функции. На изображении выше линия уровня — это линия с одинаковым цветом. По аналогии у функции могут быть линии, в каждой точке которых функция принимает одно и то же значение. Например, пусть у нас есть следующая функция:

![](https://lms.skillfactory.ru/assets/courseware/v1/3c260a7791358cc1057115d0668ec628/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_10.png)

Линии уровня для неё (если спроецировать их в нижнюю плоскость) будут выглядеть следующим образом, и, как и в примере с изменением заливки, векторы градиентов будут перпендикулярны линиям уровня:

![](https://lms.skillfactory.ru/assets/courseware/v1/fafe2b042770da62f82bd57be96be0df/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_11.png)

С линиями уровня вы уже встречались, когда видели топографические карты:

![](https://lms.skillfactory.ru/assets/courseware/v1/086cc6cc8d6d48a515621b83f57d25b4/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_12.png)

На них линиями отмечена местность с одинаковым значением высоты над уровнем моря. Причём, заметьте, что в каких-то местах линии ближе друг к другу, а в каких-то — дальше. Это происходит потому, что в каких-то местах высота увеличивается быстро (вероятно, там холмы), а в каких-то — медленно (равнины).Разумеется, если мы хотим как можно быстрее попасть повыше, нам необходимо двигаться в тех местах, где линии уровня близки друг к другу.

* Так и у функций: когда линии уровня близки друг к другу, мы понимаем, что значение функции там меняется быстро, а значит там наибольший градиент.

Итак, мы разобрались с понятием градиента функции. Для закрепления знаний пройдём ещё раз полный путь его вычисления ↓

АЛГОРИТМ нахождения градиента:

![](data/47.PNG)

Получаем, что в точке N градиент равен (2; -2), то есть при увеличении x функция будет возрастать, а при увеличении y — убывать. В точке P значение градиента равно (2;2) — это значит, что в окрестности P сумма квадратов возрастает по обеим переменным. В точке M градиент нулевой, то есть все частные производные равны нулю, а значит возможно, что в этой точке находится экстремум.

***
## **ВЫЧИСЛЕНИЕ ГРАДИЕНТА ДЛЯ ЧИСЛОВЫХ ВЕКТОРОВ В PYTHON**

Для вычисления градиента в Python есть специальная функция — gradient() из библиотеки NumPy. Градиент вычисляется на заданной N-мерной сетке с шагом, который можно задать вручную.

Такой градиент немного отличается от того, который мы обсудили ранее, поскольку это градиент для численных значений, а не для функций (хотя он обозначается так же и аналогичен по смыслу). Однако он тоже используется в DS, особенно в задачах компьютерного зрения.

Например, пусть у нас есть чёрно-белая фотография. Тогда вектор градиента может быть вычислен для каждого пикселя изображения. Это просто мера изменения значений пикселей вдоль направлений  и  вокруг каждого пикселя. То есть производная для функции характеризует скорость её изменения, а производная для численного вектора характеризует, как быстро меняются его значения — таким образом, смысл один и тот же.

Допустим, у нас есть такой фрагмент фотографии:

![](https://lms.skillfactory.ru/assets/courseware/v1/3b6ca80db274cc1d4eeb48c5a929ea87/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md5_6_14.png)

Это изображение в оттенках серого, поэтому значения пикселей варьируются от 0 до 255 (0 — чёрный, 255 — белый). Значения пикселей слева и справа от нашего пикселя отмечены на изображении: 56 и 94. Мы просто вычитаем из правого значения левое и делаем вывод, что скорость изменения в направлении x равна 38 (94 - 56 = 38).

Примечание. На данном фрагменте фотографии пиксели из тёмных становятся светлыми при перемещении слева направо. Если бы мы посмотрели на одно и то же место в другом направлении (справа налево), то получили бы отрицательное значение для изменения. Вы можете вычислить градиент, вычтя левое значение из правого или правое из левого — просто нужно вычислять градиенты одинаково на всём изображении.

Мы можем сделать то же самое и для пикселей выше и ниже, чтобы получить изменение в направлении y.

Такая возможность находить градиенты для изображения крайне важна для решения задач компьютерного зрения, так как позволяет обнаруживать границы разных объектов. Это помогает решать задачи сегментации, распознавания, классификации и многие другие.
***
Например, на изображении ниже слева вы видите изначальное изображение кошки. В центре находится градиентное изображение, в котором в каждом пикселе рассчитаны градиенты в направлении x, измеряющие горизонтальное изменение интенсивности. Справа можно видеть градиентное изображение в направлении y, измеряющее вертикальное изменение интенсивности. Серые пиксели имеют небольшой градиент, чёрные или белые пиксели — большой. В результате стали очень хорошо видны границы — это очень полезно для нахождения контуров объектов.

Если вы будете работать со свёрточными нейронными сетями для обработки изображений, то неизбежно столкнётесь с необходимостью находить такие контуры и будете использовать для этого [**фильтр Собеля**](https://habr.com/ru/post/114452/) — он как раз основан на принципе нахождения таких градиентов.
***
Итак, перейдём к примеру вычислений. Для работы с градиентом импортируем хорошо известную нам библиотеку NumPy:

In [10]:
import numpy as np
import warnings
warnings.simplefilter('ignore')
# Зададим массив, для которого хотим найти градиент, и вычислим его:

f = np.array([3, 7, 14, 23, 36, 47], dtype=np.float)
 
np.gradient(f)

array([ 4. ,  5.5,  8. , 11. , 12. , 11. ])

In [11]:
# Если специально не определять аргумент varargs, то его значение равно 1,
# так что на границах вектора мы получаем обычную разность для соседних элементов массива:

print(f[1] - f[0], f[-1] - f[-2])

4.0 11.0


In [13]:
# А вот внутри вектора для каждого элемента мы считаем разность его соседних значений,
# но уже поделённую на 2, то есть, по сути, среднее арифметическое для значений:

print((f[2] - f[0])/2)
print((f[3] - f[1])/2)
print((f[4] - f[2])/2)
print((f[5] - f[3])/2)

5.5
8.0
11.0
12.0


In [14]:
# Шаг можно варьировать. При его изменении вычисления реализуются так же,
# но деление происходит на величину обычного шага у края и удвоенную величину
# шага для элементов внутри вектора:

print(np.gradient(f, 2))
# array([2.  , 2.75, 4.  , 5.5 , 6.  , 5.5 ])

for i in range(1, len(f) - 1):
        print((f[i + 1] - f[i - 1])/(2*2))

[2.   2.75 4.   5.5  6.   5.5 ]
2.75
4.0
5.5
6.0


ВЫВОДЫ:

* Вектор градиента ортогонален линиям уровня функции.
* Градиент показывает направление наискорейшего роста функции.
* Чем плотнее расположены линии уровня, тем больше градиент.

In [40]:
# Вычислите градиент функции f = (5 - a - 2*b)^2 в точке (1,1)

a,b=symbols('a b')
f = (5 - a - 2*b)**2
f_diff_a = f.diff(a)
f_diff_b = f.diff(b)
print(f_diff_a)
print(f_diff_b)
sol = solve((f_diff_a, f_diff_b), (a, b))

grad_a = f_diff_a.evalf(subs={a:1,b:1})
grad_b = f_diff_b.evalf(subs={a:1,b:1})
print(sol)

grad = np.array([grad_a,grad_b])
grad

2*a + 4*b - 10
4*a + 8*b - 20
{a: 5 - 2*b}


array([-4.00000000000000, -8.00000000000000], dtype=object)

In [46]:
# функция имеет вид ...  Сколько решений имеет СЛАУ grad(L)=0

w0,w1=symbols('w0 w1')
L = (2.1-w0-w1)**2+(2.9-w0-3*w1)**2+(4.1-w0-5*w1)**2
L_diff_w0 = L.diff(w0)
L_diff_w1 = L.diff(w1)
print(L_diff_w0)
print(L_diff_w1)
sol = solve((L_diff_w0, L_diff_w1), (w0, w1))
sol

6.0*w0 + 18.0*w1 - 18.2
18.0*w0 + 70.0*w1 - 62.6


{w0: 1.53333333333333, w1: 0.500000000000000}

In [48]:
# Пусть дан одномерный массив f = np.array([8, 2, 8, 3, 5, 6, 5, 15]).
# Вычислите градиент массива f, увеличив шаг сетки в семь раз,
# и укажите значение седьмого элемента массива получившихся градиентов .

f = np.array([8, 2, 8, 3, 5, 6, 5, 15], dtype=np.float)
 
np.gradient(f,7)[7]

1.4285714285714286